## Code For IS445 Homework6

In [3]:
import pandas as pd
import altair as alt

data_url = 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv'
data = pd.read_csv(data_url)

In [5]:
# print(data.dtypes)

Agency Name                object
Location Name              object
Address                    object
City                       object
Zip code                    int64
County                     object
Congress Dist               int64
Congressional Full Name    object
Rep Dist                    int64
Rep Full Name              object
Senate Dist                 int64
Senator Full Name          object
Bldg Status                object
Year Acquired               int64
Year Constructed            int64
Square Footage              int64
Total Floors                int64
Floors Above Grade          int64
Floors Below Grade          int64
Usage Description          object
Usage Description 2        object
Usage Description 3        object
dtype: object


In [4]:
# print(data["Bldg Status"].unique())

['In Use' 'Abandon' 'In Progress']


In [19]:
# Group By Year and Building Status
df = data[["Year Constructed", "Bldg Status"]].dropna()
df["Year Constructed"] = df["Year Constructed"].astype(int)  

# Divide into groups based on year
df["Decade"] = (df["Year Constructed"] // 10) * 10  

# Calculate the total buildings
total_count = df.groupby("Decade").size().reset_index(name="Total Count")

# Calculate the buildings In Use
in_use_count = df[df["Bldg Status"] == "In Use"].groupby("Decade").size().reset_index(name="In Use Count")

agg_df = pd.merge(total_count, in_use_count, on="Decade", how="left")
# Calculate the proportion of building In Use
agg_df["In Use Proportion"] = agg_df["In Use Count"] / agg_df["Total Count"]  


bar_chart = (
    alt.Chart(agg_df)
    .mark_bar(color="steelblue")
    .encode(
        x=alt.X("Decade:O", title="Decade"),
        y=alt.Y("Total Count:Q", title="Total Buildings"),
        tooltip=["Decade", "Total Count"]
    )
)

line_chart = (
    alt.Chart(agg_df)
    .mark_line(color="red", point=alt.OverlayMarkDef(color="red"))
    .encode(
        x=alt.X("Decade:O", title="Decade"),
        y=alt.Y("In Use Proportion:Q", title="In Use Proportion", scale=alt.Scale(domain=[0, 1])),
        tooltip=["Decade", "In Use Proportion"]
    )
)

combined_chart = (
    alt.layer(bar_chart, line_chart)
    .resolve_scale(y="independent")  
    .properties(
        title="Total Buildings and 'In Use' Proportion by Decade",
        width=800,
        height=400,
    )
)


# combined_chart


alt.LayerChart(...)

In [52]:
data_url = 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv'
data = pd.read_csv(data_url)


df = data[["Year Constructed", "County", "Square Footage", "Total Floors"]].dropna()


agg_df = df.groupby(["Year Constructed", "County"]).agg(
    building_count=('Square Footage', 'size'),
    avg_square_footage=('Square Footage', 'mean'),
    avg_total_floors=('Total Floors', 'mean')
).reset_index()


county_selection = alt.selection_single(
    fields=['County'], 
    bind=alt.binding_select(options=agg_df['County'].unique().tolist(), name="Select County:")
)

chart = alt.Chart(agg_df).transform_filter(county_selection).mark_line().encode(
    x=alt.X('Year Constructed:O', title="Year"),
    y=alt.Y('building_count:Q', title="Building Count"),
    color=alt.Color('County:N', legend=None),
    tooltip=['Year Constructed', 'building_count', 'avg_square_footage', 'avg_total_floors']
).properties(
    title="Building Statistics Over Time"
)


chart_avg_sqft = alt.Chart(agg_df).transform_filter(county_selection).mark_line().encode(
    x=alt.X('Year Constructed:O', title="Year"),
    y=alt.Y('avg_square_footage:Q', title="Average Square Footage"),
    color=alt.Color('County:N', legend=None),
    tooltip=['Year Constructed', 'avg_square_footage']
)


chart_avg_floors = alt.Chart(agg_df).transform_filter(county_selection).mark_line().encode(
    x=alt.X('Year Constructed:O', title="Year"),
    y=alt.Y('avg_total_floors:Q', title="Average Total Floors"),
    color=alt.Color('County:N', legend=None),
    tooltip=['Year Constructed', 'avg_total_floors']
)



final_chart = alt.vconcat(
    chart, 
    chart_avg_sqft, 
    chart_avg_floors
).add_selection(
    county_selection
)


/opt/conda/lib/python3.8/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/opt/conda/lib/python3.8/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


In [53]:
# Save charts as JSON
final_chart.save('chart1.json')
combined_chart.save('chart2.json')